## Installing COBRApy

In [ ]:
!pip install cobra -q

## Loading the iML1515 Model

In [ ]:
import cobra
# Loading the E coli iML1515 Model
model = cobra.io.load_model("iML1515")

## Adding the Lycopene Pathway Missing Metabolites

In [ ]:
from cobra import Metabolite

# Create GGPP (Geranylgeranyl diphosphate)
# FPP (C15) + IPP (C5)
# Formula is C20H33O7P2
ggpp = Metabolite(
    id='ggpp_c',
    formula='C20H33O7P2',
    name='Geranylgeranyl diphosphate',
    compartment='c'
)

# Create Phytoene
# GGPP (C20) + GGPP (C20)
# Formula is C40H64.
phytoene = Metabolite(
    id='phytoene_c',
    formula='C40H64',
    name='Phytoene',
    compartment='c'
)

# Create Lycopene
# Desaturated Phytoene (lost some hydrogen)
# Formula is C40H56
lycopene = Metabolite(
    id='lycopene_c',
    formula='C40H56',
    name='Lycopene',
    compartment='c'
)

# Add the Metabolites to the model
model.add_metabolites([ggpp, phytoene, lycopene])

## Loading the Existing ones

In [ ]:
fpp = model.metabolites.get_by_id("frdp_c")  # Farnesyl diphosphate (C15)
ipp = model.metabolites.get_by_id("ipdp_c")  # Isopentenyl diphosphate (C5)
ppi = model.metabolites.get_by_id("ppi_c")   # Pyrophosphate (Byproduct)
q8  = model.metabolites.get_by_id("q8_c")    # Ubiquinone-8 (Oxidized)
q8h2= model.metabolites.get_by_id("q8h2_c")  # Ubiquinol-8 (Reduced)

## Adding the Lycopene Pathway Missing Reactions

In [ ]:
from cobra import Reaction

# Reaction 1: GGPP Synthase (CrtE)
# FPP + IPP -> GGPP + PPi
crtE = Reaction('CRTE')
crtE.name = 'GGPP Synthase'
crtE.lower_bound = -1000 
crtE.upper_bound = 1000
crtE.add_metabolites({
    fpp: -1,  
    ipp: -1,  
    ggpp: 1,  
    ppi: 1    
})

# Reaction 2: Phytoene Synthase (CrtB)
# 2 GGPP -> Phytoene + 2 PPi
crtB = Reaction('CRTB')
crtB.name = 'Phytoene Synthase'
crtB.lower_bound = -1000
crtB.upper_bound = 1000
crtB.add_metabolites({
    ggpp: -2,    
    phytoene: 1, 
    ppi: 2       
})

# Reaction 3: Phytoene Desaturase (CrtI)
# Phytoene + 4 Quinone -> Lycopene + 4 Quinol
crtI = Reaction('CRTI')
crtI.name = 'Phytoene Desaturase'
crtI.lower_bound = 0    
crtI.upper_bound = 1000
crtI.add_metabolites({
    phytoene: -1, 
    q8: -4,       
    lycopene: 1,  
    q8h2: 4       
})

# Add the Reactions to the model
model.add_reactions([crtE, crtB, crtI])

In [ ]:
# Create the demand? Reaction for accumulation/harvesting.

lycopene_demand = model.add_boundary(model.metabolites.get_by_id("lycopene_c"), type="demand")

print(f"Lycopene demand Reaction ID: {lycopene_demand.id}")

## Calculating the Max Theoritical Yield

In [ ]:
# Calculate the Max Theoritical Yield
# Change Objective to Lycopene Producton
model.objective = lycopene_demand.id
biomassID = "BIOMASS_Ec_iML1515_core_75p37M"

solution_lyco = model.optimize()
print(f"Max Lycopene Production: {solution_lyco.objective_value:.4f} mmol/gDW/h")

biomass_flux = solution_lyco.fluxes['BIOMASS_Ec_iML1515_core_75p37M']
print(f"Growth Rate during production: {biomass_flux:.4f} 1/h")

# Change Objective Back to biomass
model.objective = biomassID

## Creating the Production Envelope

In [ ]:
from cobra.flux_analysis import production_envelope

# Calculate the Production Envelope
prod_env = production_envelope(
    model,
    [lycopene_demand.id],
    objective=biomassID,
    carbon_sources="EX_glc__D_e"
)

print(prod_env.head())

In [ ]:
import matplotlib.pyplot as plt

# X-Axis: Growth Rate (flux_maximum)
# Y-Axis: Lycopene Production (DM_lycopene_c)

prod_env.plot(
    kind='line',
    x='flux_maximum',
    y=lycopene_demand.id,
    legend=False,
    color='red', 
    linewidth=2
)

plt.title('Growth vs. Lycopene')
plt.xlabel('Growth Rate (1/h)')
plt.ylabel('Max Lycopene Production (mmol/gDW/h)')
plt.grid(True, alpha=0.3)

plt.xlim(0, 1.0)
plt.ylim(0, 1.0)

plt.show()

## Generating the List of Candidates for Growth Coupling Knockouts

In [ ]:
import cobra
from cobra.flux_analysis import find_essential_reactions

model.optimize()
# Find Active Reactions
active_reactions = [r.id for r in model.reactions if abs(r.flux) > 1e-5]
print(f"Active Reactions: {len(active_reactions)}")

# Find Essential Reactions 
ess_rxns = find_essential_reactions(model)
essential_ids = set(r.id for r in ess_rxns)
print(f"Essential Reactions: {len(essential_ids)}")

# filter to find Active non essential reactions
candidates = [rid for rid in active_reactions if rid not in essential_ids]

# filter again to remove Exchange/Transport reactions 
candidates = [rid for rid in candidates if not rid.startswith("EX_") and not rid.endswith("_tpp")]

print(f"Final Candidate Count: {len(candidates)}")

## Brute Force Attempt to do a triple knockout for the selected reactions list - cuz optknock didnt work for me :'( 

In [ ]:
import cobra
import itertools
from tqdm.notebook import tqdm
import sys

biomass_id = "BIOMASS_Ec_iML1515_core_75p37M" 
target_id = lycopene_demand.id 
wild_type_growth = model.slim_optimize()

# Check: How many combos will we have to go through?
combos = list(itertools.combinations(candidates, 3)) # Around 608K 
print(f"Wild Type Growth: {wild_type_growth:.4f}")
print(f"Testing {len(combos)} combinations...")

results = []

print("STARTING BRUTE FORCE")

for r1, r2, r3 in tqdm(combos):
    
    # Get reactions
    rxn1 = model.reactions.get_by_id(r1)
    rxn2 = model.reactions.get_by_id(r2)
    rxn3 = model.reactions.get_by_id(r3)
    
    # Save Old Bounds
    old_bounds_1 = rxn1.bounds
    old_bounds_2 = rxn2.bounds
    old_bounds_3 = rxn3.bounds
    
    # Apply Knockouts
    rxn1.bounds = (0, 0)
    rxn2.bounds = (0, 0)
    rxn3.bounds = (0, 0)
    
    # Check Growth
    growth = model.slim_optimize()
    
    if growth > (0.1 * wild_type_growth):
        
        
        # Store original biomass bounds
        bm_rxn = model.reactions.get_by_id(biomass_id)
        old_bm_bounds = bm_rxn.bounds
        
        # Fix growth to 99%
        bm_rxn.bounds = (growth * 0.99, 1000)
        
        # Swap Objective to MINIMIZE Product
        model.objective = target_id
        model.objective_direction = 'min'
        
        min_prod = model.slim_optimize(error_value=0.0)
        
        # Reset Objectives & biomass bounds
        model.objective_direction = 'max'
        model.objective = biomass_id
        bm_rxn.bounds = old_bm_bounds

        # Save if Lycopene is produced
        if min_prod > 0.001:
            print(f"MATCH: {r1}, {r2}, {r3} | MinProd: {min_prod:.4f}")
            results.append({
                "Knockouts": (r1, r2, r3),
                "Growth": growth,
                "Min_Product": min_prod
            })
    
    # Revert Knockouts
    rxn1.bounds = old_bounds_1
    rxn2.bounds = old_bounds_2
    rxn3.bounds = old_bounds_3

print(f"\nFound {len(results)} Coupled Designs!")